In [ ]:
import cv2
import numpy as np
import openpifpaf
import torch

# Path to your video file
video_path = '../input/video/right_profile_1.mov'

if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple MPS (Metal Performance Shaders)")
else:
    device = torch.device('cpu')
    print("Using CPU")

# Create a Predictor instance with the desired model checkpoint
predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')

# Define the COCO keypoint connections (skeleton)
SKELETON_CONNECTIONS = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # Head to shoulders and shoulders to elbows
    (3, 5), (4, 6),  # Elbows to wrists
    (5, 7), (7, 9), (6, 8), (8, 10),  # Arms to hands
    (5, 11), (6, 12),  # Arms to hips
    (11, 13), (12, 14),  # Hips to knees
    (13, 15), (14, 16)  # Knees to ankles
]

# Open the video file using OpenCV
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Could not open video {video_path}")
    exit(1)
    
# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Original video FPS: {fps}")

# Define how many frames to skip
# For example, to process 1 out of every 5 frames:
frame_skip = 5
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Process only every Nth frame (where N = frame_skip)
    if frame_count % frame_skip == 0:
        # OpenPifPaf expects an RGB image (OpenCV loads images in BGR format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Run the predictor on the current frame
        predictions, _, _ = predictor.numpy_image(rgb_frame)
        
        # Draw the detected poses on the original frame
        for ann in predictions:
            keypoints = ann.data
            
            # Draw keypoints
            for x, y, conf in keypoints:
                if conf > 0.5:  # Only draw keypoints above a confidence threshold
                    cv2.circle(frame, (int(x), int(y)), radius=5, color=(0, 255, 0), thickness=-1)
            
            # Draw skeleton (connecting lines between keypoints)
            for p1_idx, p2_idx in SKELETON_CONNECTIONS:
                if (keypoints[p1_idx, 2] > 0.5 and keypoints[p2_idx, 2] > 0.5):
                    p1 = (int(keypoints[p1_idx, 0]), int(keypoints[p1_idx, 1]))
                    p2 = (int(keypoints[p2_idx, 0]), int(keypoints[p2_idx, 1]))
                    cv2.line(frame, p1, p2, color=(0, 0, 255), thickness=2)
        
        # Display the frame with pose estimates
        cv2.imshow('OpenPifPaf Pose Estimation', frame)
        
        # Control display speed (adjust the wait time if needed)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break
    
    # Increment frame counter
    frame_count += 1

# Release resources
cap.release()
cv2.destroyAllWindows()

Using Apple MPS (Metal Performance Shaders)
Original video FPS: 25.153309367731023


2025-04-16 09:57:15.834 Python[93929:9005138] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-16 09:57:15.834 Python[93929:9005138] +[IMKInputSession subclass]: chose IMKInputSession_Modern
